In [ ]:
import gpxpy # Анализ gpx файлов
import gpxpy.gpx 
import os
import datetime
import speech_recognition as sr # Распознование голоса
from docx import Document # Docx документы
from docx.shared import Pt
from pydub import AudioSegment #
from exif import Image # Парсер Расширенного формат файлов
import pytz # Часовые пояса

r = sr.Recognizer() # Экземпляр распознователя голоса
d = Document() # Word-документ с хронометражом 

tz = pytz.timezone("Europe/Moscow") # Часовой пояс
gpx_file = open('track.gpx', 'r') # Относительный адресс файла с треком
path_dict = 'records' # Относительный адресс непустой папки с диктофонными записями
path_photo = 'photos' # Относительный адресс непустой папки с фото
wit_key = "" # Токен сервиса по распознованию голоса. Выдаётся на wit.ai.
result = "report.docx" # Файл для сохранения хронометража

files = os.listdir(path_dict)
files = [os.path.join(path_dict, file) for file in files]
files = [file for file in files if os.path.isfile(file)]
files.sort(key=os.path.getmtime) # Диктофонные записи

photo = os.listdir(path_photo)
photo = [os.path.join(path_photo, file) for file in photo]
photo = [file for file in photo if os.path.isfile(file)]
photo = list(map((lambda f: (f, Image(open(f, 'rb')).datetime_original)), photo))
photo.sort(key=(lambda t: t[1])) #Фотографии

radial = [(5,2.4), (8,1.9)] # Радиалки в формате (номер дня, длинна в км)
totalrad = 0 # Итого радиально

skip = [] # Днёвки в формате номеров дней - из километража исключаются

gpx = gpxpy.parse(gpx_file) # Трек


prpoint = None # Предыдущая точка
dbpoint = None # Точка начала дня

dayn = 0 # Счётчик дней
delay = 300 # Минимальное время остановки в секундах. 
            # Если время между двумя точками превышает этот интервал - то это отмечается в хронометраже
nd_hour = 3 # В три часа ночи начинается трек нового дня
    
dist = 0 # Глобальная дистанция
ddist = 0 # Дневная дистанция
high = 0 # Глобальный набор высоты 
dhigh = 0 # Дневной набор высоты
low = 0 # Глобальный сброс высоты
dlow = 0 # Дневной сброс высоты

# Широту/Долготу в строку
def get_cs(coord):
    return "{0:}° {1}' {2:.2f}''".format(round(coord),round(coord*60)%60,round(coord*3600,2)%60)

# время следущей диктофонной
def dicttime():
    return datetime.datetime.fromtimestamp(os.path.getmtime(files[0]))

# Точку в строку
def get_csp(point):
    return "{0} {1}, {2} {3}".format('N' if point.longitude>=0 else 'S',get_cs(point.longitude),
                                   'E' if point.latitude>=0 else 'W',get_cs(point.latitude))
def zapis(n):
    if 10 < n%100 < 20 or 4 < n%10 < 9 or n%10 == 0:
        return "записей"
    if 1 < n%10 < 5:
        return "записи"
    return "запись"

# минуты
def minute(p):
    return p.time.astimezone(tz).minute

# часы
def hour(p):
    return p.time.astimezone(tz).hour

# Распознать аудиозапись
def getdictstr_rec(audio):
    try:
        return r.recognize_wit(audio, key=wit_key)
    except sr.UnknownValueError:
        return "Запись"
    except sr.RequestError as e:
        #print(e)
        return str(e)
        #return getdictstr_rec(audio)

# Диктофонную запись в строку
def getdictstr(file):
    wma = AudioSegment.from_file(os.path.abspath(file), "asf")
    wma.export("temp.wav", format="wav")
    with sr.AudioFile("temp.wav") as source:
        audio = r.record(source)
    return getdictstr_rec(audio)
        
# Печать в word
def wordprint(s, b=False, c=False):
    print(s)
    p = d.add_paragraph()
    r = p.add_run(s)
    r.bold = b
    r.italic = c
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    return p

# Альтернативная печать в word
def wordprint2(s1, s2):
    p = wordprint(s1,b=True)
    r = p.add_run(s2)
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    print(s2)
    return p

# Добавить в хронометраж диктофонные записи к данной точке
def printdict(p, f_all=True):
    if len(files) > 0:
        tm = dicttime()
        while (len(files) > 0 
               and tz.localize(tm) < p.time.astimezone(tz) 
               and (f_all or tm.hour >= nd_hour 
                    and tm.day == p.time.astimezone(tz).day-1 
                    or tm.hour < nd_hour 
                    and tm.day == p.time.astimezone(tz).day)):
            dictstr = getdictstr(files[0])#"Запись."
            wordprint2("{0}:{1:02d}\u00A0({2:.2f}\u00A0км/{3:.2f}\u00A0км)\u00A0"
                       .format(tm.hour,tm.minute,ddist/1000.0,dist/1000.0),dictstr)
            #print(files[0])
            files.remove(files[0])
            if len(files) > 0:
                tm = dicttime()
    return

# Добавить в хронометраж имена фото-файлов к данной точке
def printphoto(p, f_all=True):
    if len(photo) > 0:
        tm = datetime.datetime.strptime(photo[0][1], '%Y:%m:%d %H:%M:%S')
        while (len(photo) > 0 
               and tz.localize(tm) < p.time.astimezone(tz) 
               and (f_all or tm.hour >= nd_hour 
                    and tm.day == p.time.astimezone(tz).day-1 
                    or tm.hour < nd_hour
                    and tm.day == p.time.astimezone(tz).day)):
            dictstr = os.path.basename(photo[0][0])
            wordprint2("{0}:{1:02d}\u00A0({2:.2f}\u00A0км/{3:.2f}\u00A0км)\u00A0"
                       .format(tm.hour,tm.minute,ddist/1000.0,dist/1000.0),dictstr)
            photo.remove(photo[0])
            if len(photo) > 0:
                tm = datetime.datetime.strptime(photo[0][1], '%Y:%m:%d %H:%M:%S')
    return

# Вернуть диктофонные записи и их число в данной точке
def dictn(p):
    count = 0
    dictstr = ""
    if len(files) > 0:
        tm = dicttime()
        while len(files) > 0 and tz.localize(tm) < p.time.astimezone(tz):
            dictstr += getdictstr(files[0]) + ". "
            files.remove(files[0])
            count+=1
            if len(files) > 0:
                tm = dicttime()
    return (dictstr, count) #(str(count) + " " + zapis(count) + ".", count)

# Вернуть имена файлов с фото и их число в данной точке
def photon(p):
    count = 0
    phstr = ""
    if len(photo) > 0:
        tm = datetime.datetime.strptime(photo[0][1], '%Y:%m:%d %H:%M:%S')
        while len(photo) > 0 and tz.localize(tm) < p.time.astimezone(tz):
            phstr+= ("" if phstr == "" else " ") + os.path.basename(photo[0][0])
            count+=1
            photo.remove(photo[0])
            if len(photo) > 0:
                tm = datetime.datetime.strptime(photo[0][1], '%Y:%m:%d %H:%M:%S')
    return (phstr + ".", count)

# Печать фото и диктофона к данной точке
def printdnf(p, f_all=True):
    printdict(p, f_all = f_all)
    printphoto(p, f_all = f_all)
    return

# dictn + photon
def dnfn(p):
    (t0, t1) = dictn(p)
    (t2, t3) = photon(p)
    return (t0, t1, t2, t3)

for track in gpx.tracks:
    for segment in track.segments:
        
        # Чистим трек от точек проставленных на скорости <= 2 км/ч
        pns = list()
        pn = 0
        for p1, p2 in zip(segment.points[:-1], segment.points[1:]):
            if (p1.distance_3d(p2)/p1.time_difference(p2)*3.6) <= 2.0:
                pns.append(pn)
            pn+=1
        for p in pns[::-1]:
            segment.remove_point(p)
        
        # Обработка дней похода
        for point in segment.points:
            if prpoint == None:
                prpoint = point
            if ((dbpoint == None) 
                or (point.time.astimezone(tz).day != dbpoint.time.astimezone(tz).day 
                    and hour(point) >= nd_hour)):
                dayn+=1
                if dbpoint != None:
                    wordprint2("{0}:{1:02d}\u00A0({2:.2f}\u00A0км/{3:.2f}\u00A0км)\u00A0"
                               .format(hour(prpoint),minute(prpoint),ddist/1000.0,dist/1000.0), "Финиш.")
                    printdnf(point, f_all = False)
                    wordprint("Пройденное расстояние: {0:.2f} км".format(ddist/1000.0), c=True)
                    if len(radial) > 0 and radial[0][0] == dayn - 1:
                        wordprint("из них радиально: {0:.2f} км".format(radial[0][1]), c=True)
                        totalrad += radial[0][1]
                        radial.remove(radial[0])
                    wordprint("Набор высоты: {0:.1f} м".format(dhigh), c=True)
                    wordprint("Сброс высоты: {0:.1f} м".format(dlow), c=True)
                    wordprint("Высота ночёвки: {0:.1f} м".format(point.elevation), c=True)
                    wordprint("Координаты точки ночёвки: {0}".format(get_csp(point)), c=True)
                    wordprint("")
                else:
                    printdnf(point)
                    wordprint("")
                wordprint("День {0}. {1}".format(dayn,point.time.astimezone(tz).date()),b=True,c=True)
                wordprint("МС{0} \u2014 МС{1}".format(dayn-1,dayn),c=True)
                if dayn in skip:
                    wordprint("Днёвка.")
                dbpoint = point
                ddist = 0
                dhigh = 0
                dlow = 0
                printdnf(point)
                wordprint2("{0}:{1:02d}\u00A0({2:.2f}\u00A0км/{3:.2f}\u00A0км)\u00A0".format
                    (hour(point),minute(point),ddist/1000.0,dist/1000.0), "Старт.")
            else:
                if not dayn in skip:
                    dist += prpoint.distance_3d(point)
                    if point.elevation-prpoint.elevation > 0:
                        high += point.elevation-prpoint.elevation
                    else:
                        low += prpoint.elevation-point.elevation
                ddist += point.distance_3d(prpoint)
                if point.elevation-prpoint.elevation > 0:
                        dhigh += point.elevation-prpoint.elevation
                else:
                        dlow += prpoint.elevation-point.elevation
                if point.time_difference(prpoint) > delay:
                    (dicts, x, photos, y) = dnfn(point)
                    wordprint2("{0}:{1:02d}-{2}:{3:02d}\u00A0({4:.2f}\u00A0км/{5:.2f}\u00A0км)\u00A0"
                               .format(hour(prpoint),minute(prpoint),hour(point),
                                       minute(point),ddist/1000.0,dist/1000.0),
                                       ("" if x == 0 else dicts)+("" if y == 0 else photos))
                else:
                    printdnf(point)
            prpoint = point

# Последний день с итогами
if dbpoint != None:
    wordprint2("{0}:{1:02d}\u00A0({2:.2f}/{3:.2f})\u00A0"
               .format(hour(point),minute(point),ddist/1000.0,dist/1000.0),"Финиш.")
    wordprint("Пройденное расстояние: {0:.2f} км".format(ddist/1000.0), c=True)
    if len(radial) > 0 and radial[0][0] == dayn:
        wordprint("из них радиально: {0:.2f} км".format(radial[0][1]), c=True)
        totalrad += radial[0][1]
        radial.remove(radial[0])
    wordprint("Набор высоты: {0:.1f} м".format(dhigh), c=True)
    wordprint("Сброс высоты: {0:.1f} м".format(dlow), c=True)
    wordprint("Высота ночёвки: {0:.1f} м".format(point.elevation), c=True)
    wordprint("Координаты точки ночёвки: {0}, {1}".format(get_cs(point.latitude),get_cs(point.longitude)), c=True)
    wordprint("")
    wordprint("\nИтого:", c=True)
    wordprint("    Дней: {0}".format(dayn), c=True)
    wordprint("    Пройденное расстояние: {0:.2f} км".format(dist/1000.0), c=True)
    wordprint("    Из них радиально: {0:.2f} км".format(totalrad), c=True)
    wordprint("    Набор высоты: {0:.1f} м".format(high), c=True)
    wordprint("    Сброс высоты: {0:.1f} м".format(low), c=True)

#for waypoint in gpx.waypoints:
#    print('Точка {0} -> ({1},{2})'.format(waypoint.name, waypoint.latitude, waypoint.longitude))

# Печатаем результат в файл
d.save(result)

In [ ]:
d.save("Хронометраж.docx")